In [1]:
import ete3
from ete3 import Tree
import os
import os
import glob
import numpy as np
from Bio import AlignIO 
import random
import re

In [2]:
t = Tree("(((A:0.687, B:0.601):1.244,(C:0.804,(D:0.015, E:0.017):0.79):0.333):0.555,F:1.001);")

In [3]:
t

Tree node '' (0x7fbb05302b7)

In [4]:
os.getcwd()
os.chdir("./Alignments/block_aln/")

# Getting block-alignment

In [409]:
for file in glob.glob("*.fasta"):
    with open(file, 'r') as fname:
        matrix = np.array([np.array(rec.seq) for rec in AlignIO.read(fname, "fasta")])
        names = [rec.name for rec in AlignIO.read("PF00004_1.fasta", "fasta")]
        indexes = [num for num in range(matrix.shape[1]) if list(matrix[:,num]).count("-") > len(matrix[:,num]) * 0.5]
        matrix = np.delete(matrix, indexes, axis=1)
    with open("./block_aln/"+file.split(".")[0]+"_block.fasta", 'w') as fname:
        for name, seq in zip(names, list(matrix)):
            fname.write(">" + name + '\n' + ''.join(list(seq)) + '\n')

# New way

In [5]:
class nd:
    def __init__(self, seq):
        self.seq = seq
        #self.name = name
    def children(self, children):
        self.children = children

In [263]:
class parser(nd):
    def __init__(self, x, tree):
        self.x = x
        self.t = tree
        super().__init__(0)

    #Choose random alignment (with seq_len more than self.x)
    def random_aln(self):
        while True:
            file = random.choice([file for file in glob.glob("*.fasta")])
            aln = AlignIO.read(open(file, 'r'), "fasta")
            if len(str(aln[0].seq)) >= self.x:
                break
        self.file = file
        self.alignment_dict = {record.id:str(record.seq) for record in aln}
        
    #Make root sequence and pin columns   
    def root_seq(self):
        column_id = [random.randint(0, self.x) for _ in range(self.x)]
        a = AlignIO.read(open(self.file, 'r'), "fasta")
        names, seq = [],''
        for num in range(self.x):
            record = random.randint(0, len(a)-1)
            names.append(a[record].id)
            seq += str(a[record].seq)[column_id[num]] 
        self.columns = column_id
        self.seq = seq
        self.names = names
        #print(self.columns, '\n', self.names, '\n', self.file)
             
    #Make all_dists and all_id lists
    def dists(self):
        all_id, all_dists, values, one_id = [], [], [], ''
        with open(self.file[:-12]+".dist", 'r') as fname:
            for string in fname:
                if re.search("[A-Za-z]+", string) != None:
                    all_id.append(one_id)
                    all_dists.append(values)
                    one_id, values = string.split()[0], string.split()[1:]
                else:
                    values += string.split()
            all_id.append(one_id)
            all_dists.append(values)
        self.all_id = all_id[1:]
        self.all_dists = all_dists[1:]
        
    #NORMALIZATION
    def normal(self):
        #print(self.all_dists[0])
        diff = abs(max([max(list(map(lambda x: float(x), group))) for group in self.all_dists])-min([min(list(map(lambda x: float(x), group))) for group in self.all_dists]))
        tree_diff = abs(max([n.dist for n in self.t.traverse()][1:]) - 0)#-min([n.dist for n in self.t.traverse()][1:]))
        alpha = diff/tree_diff
        
        for i in range(len(self.all_dists)):
            self.all_dists[i] = list(map(lambda x: str(float(x)*alpha), self.all_dists[i]))
        #print(self.all_dists[0])
    #Make sequences
    def make_sample(self):
        nodes, graph, lvl = [], [], -1
        #self.nodes.append(node('root', 0.0, self.root_seq))
        for node in self.t.traverse("levelorder"):
            for i in list(map(lambda x: x.dist, node.children)):
                graph.append([node.dist, i]) 
        self.graph = graph  
        #print(self.graph)
        lvl = 0
        while self.graph[lvl][0] == 0.0:
            self.graph[lvl][0] = self.names 
            lvl += 1     
            
        #check_point_id = self.all_id
        check_point_dists = self.all_dists
        for item in self.graph:
            self.all_dists = check_point_dists
            self.names = item[0]
            dist = float(item[1])
            seq = ''
            names = []
            less, more, less_index, more_index = 100, 100, [], [] #0, 0       
            
            #################################################################################

            # Generating sequence
            for al_num in range(len(self.names)):
                ##print(self.names[al_num])
                #print(self.all_id[self.all_id.index(self.names[al_num])])
                #print(self.all_id.index(self.names[al_num]))
                values = self.all_dists[self.all_id.index(self.names[al_num])]
                ##print(sorted(values))
                
                less, more = 100, 100
                for i in range(len(values)):
                    if float(values[i]) <= dist and abs(float(values[i])-dist) < abs(float(less)-dist):
                        #print(values[i], less)
                        less = values[i]
                    elif float(values[i]) > dist and abs(float(values[i])-dist) < abs(float(more)-dist):
                        more = values[i]
                our_dist = random.choice([less, more])
                print(our_dist, dist, values)
                ind = random.choice([i for i, x in enumerate(values) if x == our_dist])
                self.all_dists[self.all_dists.index(values)][ind] = '1000'
                names.append(self.all_id[ind])
                seq += self.alignment_dict[self.all_id[ind]][self.columns[al_num]]
            
            
            
            print(seq)
            i = 0
            while True:
                if i == len(graph):
                    break
                if self.graph[i][0] == dist:
                    break
                i += 1
            while True:
                if i == len(graph):
                    break
                if self.graph[i][0] != dist:
                    break
                if self.graph[i][0] == dist:
                    self.graph[i][0] = names
                i += 1
            #print(seq, item[1])
            nodes.append(nd(seq))
        self.sample = nodes

In [266]:
f = parser(100, t)
f.random_aln()
f.file
f.root_seq()
f.seq
f.dists()
#f.normal()
f.seq
f.make_sample()

0.537177 0.555 ['1.855741', '1.477776', '0.967085', '0.769370', '0.586533', '0.578203', '0.661582', '0.766232', '0.681259', '0.743728', '0.806513', '0.754262', '0.784159', '0.629228', '0.579100', '0.596976', '0.651144', '0.537177', '0.605341', '0.587391', '0.496280', '0.667268', '0.533848', '0.570028', '0.377124', '0.262863', '0.268278', '0.296541', '0.239973', '0.256861', '0.211405', '0.287878', '0.230057', '0.219263', '0.211159', '0.229402', '0.228504', '0.119897', '0.076694', '0.000000', '0.223123', '0.279319', '0.294305', '0.274688', '0.286944', '0.224213', '0.196331', '0.272259', '0.275136', '0.266691']
0.533848 0.555 ['1.855741', '1.477776', '0.967085', '0.769370', '0.586533', '0.578203', '0.661582', '0.766232', '0.681259', '0.743728', '0.806513', '0.754262', '0.784159', '0.629228', '0.579100', '0.596976', '0.651144', '1000', '0.605341', '0.587391', '0.496280', '0.667268', '0.533848', '0.570028', '0.377124', '0.262863', '0.268278', '0.296541', '0.239973', '0.256861', '0.211405', 

IndexError: Cannot choose from an empty sequence

In [205]:
f.file

'PF00164_1_block.fasta'